In [1]:

from app.log import logging_config
from dotenv import load_dotenv
import logging
from app.util.message import starting_message
from app import Polygon
from app import Sentiment_predict
import pandas as pd
from app.api import Twitter 
from os import environ


c:\Users\andre\Anaconda3\envs\DS4A_PF\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Polygon

In [2]:
##Load and upgrade initial df
polygon_prev = pd.read_csv("polygon_df_01_07_2022.csv")
#Polygon credentials
load_dotenv()
print(environ.get("POLYGON_KEY"))
logging_config.init_logging()
__LOG = logging.getLogger(__name__)
__LOG.info('...... Initialization Completed  ......')
__LOG.info(starting_message())
#Initialize Polygon
polygon = Polygon(environ.get("POLYGON_KEY"))
#Download News
status, polygon_data = polygon.get_news('')
if status:
    polygon_news_df = pd.DataFrame.from_dict(polygon_data['results'])
polygon_news_df.drop(['publisher', 'keywords'], axis = 1, inplace = True)
polygon_news_df['tickers'] = [' '.join(map(str, l)) for l in polygon_news_df['tickers']]
#predict Sentiment
sp = Sentiment_predict()
polygon_news_df['title_finBERT'] = sp.sentiment_finBERT(polygon_news_df, 'title')
polygon_news_df['title_financialBERT'] = sp.sentiment_financialBERT(polygon_news_df, 'title')
polygon_news_df['description_finBERT'] = sp.sentiment_finBERT(polygon_news_df, 'description')
polygon_news_df['description_financialBERT'] = sp.sentiment_financialBERT(polygon_news_df, 'description')
#Upload df.
polygon_news_df = pd.concat([polygon_news_df, polygon_prev], ignore_index = True).drop_duplicates().reset_index(drop = True)
polygon_news_df.to_csv("polygon_df_01_07_2022.csv", index = False, encoding='utf-8')



PWRjSfmBc9xYnPZMQhA9ukDOuOIId90D


Token indices sequence length is longer than the specified maximum sequence length for this model (774 > 512). Running this sequence through the model will result in indexing errors


in row 752 the text is divided into two parts
in row 997 the text is divided into two parts


Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


in row 752 the text is divided into two parts
in row 997 the text is divided into two parts


In [3]:
polygon_news_df.shape

(1345, 13)

In [4]:
#Polygon plots Data
df_poly = polygon_news_df[['id','published_utc', 'title','description', 'title_finBERT', 'description_finBERT', 'title_financialBERT', 'description_financialBERT', 'article_url']]
df_poly['date'] = df_poly['published_utc'].str[0:10]
df_poly["total_news_day"] = df_poly.groupby('date')['id'].transform(len)
df_poly = df_poly.groupby(['date','title_finBERT']).size().reset_index()
df_poly.rename({0:'count'}, axis = 1, inplace = True)
df_poly['date'] = pd.to_datetime(df_poly['date'], format='%Y-%m-%d')
df_poly

C:\Users\andre\AppData\Local\Temp\ipykernel_31892\1300864731.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_poly['date'] = df_poly['published_utc'].str[0:10]
C:\Users\andre\AppData\Local\Temp\ipykernel_31892\1300864731.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_poly["total_news_day"] = df_poly.groupby('date')['id'].transform(len)


,date,title_finBERT,count
0,2022-06-30,negative,87
1,2022-06-30,neutral,312
2,2022-06-30,positive,80
3,2022-07-01,negative,123
4,2022-07-01,neutral,464
5,2022-07-01,positive,98
6,2022-07-02,negative,29
7,2022-07-02,neutral,84
8,2022-07-02,positive,15
9,2022-07-03,negative,10


In [5]:
df_poly.dtypes

date             datetime64[ns]
title_finBERT            object
count                     int64
dtype: object

## Tweets

In [6]:
twt = Twitter()
df_twt = twt.get_tweets_df(ticker = 'BTC', query = 'bitcoin OR BTC AND crypto', limit = 100, popular =True)
df_twt['tweet_finBERT'] = sp.sentiment_finBERT(df_twt, 'full_text')
df_twt['tweet_financialBERT'] = sp.sentiment_financialBERT(df_twt, 'full_text')
df_twt.dtypes

id                     object
created_at             object
full_text              object
truncated                bool
retweets                int64
user_name              object
user_verified            bool
user_followers          int64
user_image_url         object
ticker                 object
tweet_finBERT          object
tweet_financialBERT    object
dtype: object

In [7]:
df_twt

,id,created_at,full_text,truncated,retweets,user_name,user_verified,user_followers,user_image_url,ticker,tweet_finBERT,tweet_financialBERT
0,1543591384602533888,Sun Jul 03 13:43:59 +0000 2022,"bitcoin was $45,000 and ethereum was $3000 whe...",False,337,cobie,True,713006,http://pbs.twimg.com/profile_images/1542300448...,BTC,neutral,neutral
1,1543315422740525056,Sat Jul 02 19:27:24 +0000 2022,"The El Salvador Bitcoin ""Experiment"" has not o...",False,91,ncweaver,True,24077,http://pbs.twimg.com/profile_images/1509903909...,BTC,neutral,negative
2,1543098138612973568,Sat Jul 02 05:04:00 +0000 2022,I am just going to keep stacking #bitcoin and ...,False,165,TheCryptoLark,True,1023967,http://pbs.twimg.com/profile_images/1482889740...,BTC,neutral,neutral
3,1543621772687335430,Sun Jul 03 15:44:44 +0000 2022,We have an AMA with @metfxmetaverse at 5 PM UT...,False,77,GollumsGems,True,29394,http://pbs.twimg.com/profile_images/1524397405...,BTC,neutral,positive
4,1543241259451555842,Sat Jul 02 14:32:42 +0000 2022,"Top #Bitcoin whale accumulates 4816 BTC, spend...",False,126,WhaleCoinTalk,True,125195,http://pbs.twimg.com/profile_images/1495764154...,BTC,neutral,neutral
5,1543263478827298820,Sat Jul 02 16:01:00 +0000 2022,"Blackrock, $BLK, has said #Bitcoin and crypto ...",False,83,unusual_whales,True,657481,http://pbs.twimg.com/profile_images/1479920678...,BTC,neutral,neutral
6,1543015091259555840,Fri Jul 01 23:34:00 +0000 2022,Many of the world's richest and most famous in...,False,150,TheCryptoLark,True,1023967,http://pbs.twimg.com/profile_images/1482889740...,BTC,neutral,neutral
7,1543425444615139328,Sun Jul 03 02:44:36 +0000 2022,Vote for your favorite #UFC276 athletes to win...,False,55,ufc,True,9957141,http://pbs.twimg.com/profile_images/1543467453...,BTC,neutral,positive
8,1543588682967531528,Sun Jul 03 13:33:15 +0000 2022,Pick your #crypto team:\n\nTeam 🅰️ T...,False,58,TheMoonCarl,True,1135770,http://pbs.twimg.com/profile_images/1490246864...,BTC,neutral,neutral
9,1543600482127646720,Sun Jul 03 14:20:08 +0000 2022,"In 2019, Maastricht University (Netherlands) p...",False,43,campuscodi,True,103050,http://pbs.twimg.com/profile_images/1312122090...,BTC,neutral,neutral


In [8]:
df_twt['full_text'].iloc[35]

'The prolonged slump in Bitcoin is making it more difficult for some miners to repay the up to $4 billion in loans they have backed by their equipment, posing a potential risk to major crypto lenders https://t.co/tx9VOfQl32'

In [9]:
month_number = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May': '05','Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

df_twt['day'] = df_twt['created_at'].str[8:10]
df_twt['month'] = df_twt['created_at'].str[4:7].map(month_number)
df_twt['year'] = df_twt['created_at'].str[-4:]

df_twt['date'] = df_twt['year'] + '-' + df_twt['month'] + '-' + df_twt['day'] 
#Twitter plots Data
df_twt["total_news_day"] = df_twt.groupby('date')['id'].transform(len)
twt_df = df_twt.groupby(['date', 'tweet_finBERT']).size().reset_index()
twt_df.rename({0:'count'}, axis = 1, inplace = True)
twt_df['date'] = pd.to_datetime(twt_df['date'], format='%Y-%m-%d')
twt_df

,date,tweet_finBERT,count
0,2022-06-24,neutral,8
1,2022-06-24,positive,1
2,2022-06-25,negative,3
3,2022-06-25,neutral,6
4,2022-06-27,neutral,1
5,2022-06-29,negative,1
6,2022-06-30,negative,1
7,2022-06-30,positive,1
8,2022-07-01,negative,2
9,2022-07-01,neutral,8


## Dash plots

In [10]:
import plotly.express as px
def sent_pie(df, count, col_name, plot_title):
    fig = px.pie(df, values = count, names = col_name, template='plotly_dark', title = plot_title)
    fig.update_layout(
    legend_traceorder="reversed",
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
        #family="Courier New, monospace",
        size=25)
        },
    )
    
    return fig
def sent_bar(df, date, count, col_name, plot_title):
    fig = px.bar(df, x=date, y=count, color=col_name, template='plotly_dark', category_orders= {col_name: ["neutral", "negative", "positive"]}, title= plot_title)
    fig.update_layout(
        legend_traceorder="reversed",
        legend_title = '',
        title={
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
            #family="Courier New, monospace",
            size=25)
            },
        )
    fig.layout.xaxis.dtick = 'D1'
    return fig

In [11]:
sent_pie(twt_df, 'count', 'tweet_finBERT', "Tweets by sentiment").show()

In [12]:
sent_pie(df_poly, 'count', 'title_finBERT', "News by sentiment").show()

In [13]:
sent_bar(twt_df, 'date', 'count', 'tweet_finBERT', "Daily Tweets by Sentiment").show()


In [14]:
sent_bar(df_poly, 'date', 'count', 'title_finBERT', "Daily News by Sentiment").show()